In [92]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import numpy.polynomial.polynomial as poly
from scripts.proj1_helpers import *
from implementations import *
from cross_validations import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
y, input_data, ids = load_csv_data('../../train.csv')

In [29]:
y_test, test_data, ids_test = load_csv_data('../../test.csv')

In [152]:
def cross_validation_step(y, x, degree):
    seed = 1
    #degree = 4
    k_fold = 2
    lambdas = [0.001, 0.002, 0.001, 0.001, 0.01, 0.6, 0.1]#np.logspace(-4, 0, 30)
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)
    # define lists to store the loss of training data and test data
    rmse_tr = []
    rmse_te = []
    ws = []
    # ***************************************************
    # INSERT YOUR CODE HERE
    # cross validation: TODO
    # ***************************************************
    x = np.hstack((np.ones((x.shape[0], 1)), x))
    for lm in lambdas:
        loss = np.zeros(len(y))
        w = np.zeros(x.shape[1])
        best_train_acc = 0
        best_test_acc = 0
        for i in range(k_fold):
            acc_train, acc_test = cross_validation(y, x, k_indices, i, lm, degree)
            
            if acc_train > best_train_acc or acc_test > best_test_acc:
                print('Accuracy of the method: train {}; test {}; degree {}; lambda {}; {} fold'.format(acc_train, acc_test,degree, lm,i))

def cross_validation_stepg(y, x, regression):
    seed = 1
    k_fold = 2
    max_iters = 100
    gamma = 0.01
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)
    # ***************************************************
    # INSERT YOUR CODE HERE
    # cross validation: TODO
    # ***************************************************

    #loss = np.zeros(len(y))
    x = np.hstack((np.ones((x.shape[0], 1)), x))
    w ,loss = least_squares(y,x)
    #w = np.zeros(x.shape[1])
    print('Starting')
    best_acc = 0
    for i in range(k_fold): 
        w_i, acc_train, acc_test = cross_validation_generic(y, x, regression, k_indices, i, w, max_iters, gamma)
        #loss += np.array(loss_tr, loss_te)
        if acc_train > best_acc:
            best_acc = acc_train
            #w = w_i
            print('Accuracy of the method: {} for the {}th fold'.format(acc_train,i))
            
def cross_validation_stepl(y, x, poly, degree):
    seed = 1
    k_fold = 2
    max_iters = 1000
    gamma = 1.2
    lambda_ = 0.1
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)

    x=x[:,1:]

    w = np.zeros(x.shape[1])
    best_acc = 0
    for i in range(k_fold): 
        y_pred, acc_train, acc_test = cross_validation_logistic(y, x, k_indices, i, w, max_iters, gamma, lambda_, poly, degree)
        if acc_train > best_acc:
            best_acc = acc_train
            best_pred = y_pred
        print('Accuracy of the method: {} for the {}th fold'.format(acc_train,i))
    return best_pred
        

def best_degree(y, x):
    lambdas = []
    ws = []
    rmse = []
    
    for degree in range(1, 10):
        cross_validation_step(y, x, degree)
        

def best_param_jets(y,x):
    seed = 8
    k_fold = 4
    max_iters = 200
    
    #x=x[:,1:]
    predicted_w = []
    
    w = np.zeros(x.shape[1])
    best_acc = 0
    best_average_acc = 0
    
    y_hat = np.zeros(len(y))
    y_pred_average = np.zeros(len(y))
    jets_ind = jets_indices(x)
    x = standardize(x)
    
    params = [[jets_ind[0], 0.4, 0.02, 5],
             [jets_ind[1], 0.8, 0.001, 6],
             [jets_ind[2], 0.9, 0.01, 5]]

    
    for param in [params[1]]:
        index = param[0]
        gamma = param[1]
        lambda_ = param[2]
        degree = param[3]
        x_tr = x[index[0]][:,index[1]]
        y_tr = y[index[0]]
        
        k_indices = build_k_indices(y_tr, k_fold, seed)
        
        ws = []
        best_acc =0
        for i in range(k_fold): 

            y_pred, acc_train, w = cross_validation_jets(y_tr, x_tr, k_indices, i, w, max_iters, gamma, lambda_, degree)
            ws.append(w)
            if acc_train > best_acc:
                best_acc = acc_train
                best_pred = y_pred

                print('{} | fold {}'.format(acc_train, i))
            

        average_w = np.array(ws).mean(axis=0)
        
        predicted_w.append(average_w)
        
        x_poly = build_poly(x_tr, degree)
        x_poly = np.hstack((np.ones((x_poly.shape[0], 1)), x_poly))
        average_pred = predict_labels(average_w, x_poly)            
        y_hat[index[0]] = best_pred
        y_pred_average[index[0]] = average_pred
        average_acc = accuracy(y[index[0]], average_pred)
        print(average_acc)
        
    
    #total_acc = accuracy(y, y_hat)
    #total_average_acc = accuracy(y, y_pred_average)
    
    #return predicted_w, total_acc, total_average_acc
    

In [5]:
#best_degree(y, input_data)

In [6]:
#cross_validation_stepg(y, input_data, logistic_regression_newton_method)

In [7]:
#cross_validation_stepl(y, input_data, poly=True, degree=2)

In [153]:
predicted_w, a, acc = best_param_jets(y,input_data)
print("Average accuracy {}".format(acc))

Current iteration=0, loss=40312.053727005296, acc=0.8018501323979504, gamma=0.8, lambda=0.001
Current iteration=50, loss=25663.855229617537, acc=0.8122356339626534, gamma=0.8, lambda=0.001
Current iteration=100, loss=25358.204774880807, acc=0.8118573541043365, gamma=0.8, lambda=0.001
Current iteration=150, loss=25280.701077581864, acc=0.8191306441074315, gamma=0.8, lambda=0.001
0.8216926304205784 | fold 0
Current iteration=0, loss=40312.053727005296, acc=0.8016266033907631, gamma=0.8, lambda=0.001


KeyboardInterrupt: 

In [28]:
predicted_w

[array([ -3.10149341e+03,   1.95527752e+04,   2.14784524e+04,
         -3.39889027e+03,   1.84688985e+04,   9.82306003e+02,
         -2.65288836e+03,   1.71842539e+04,  -1.09070867e+04,
         -3.13629257e+03,  -1.03569846e+04,   1.74942684e+04,
          1.67698454e+04,   5.08991853e+03,  -2.19351558e+04,
          3.97698439e+03,  -4.85626740e+03,  -1.35761202e+03,
          7.56625848e+03,  -7.24777304e-02,   2.70513291e-01,
          1.50829059e+04,   1.95656062e-01,  -5.22324346e+03,
          1.95909177e+04,   2.30241460e-01,  -1.88472312e-01,
         -6.88817357e+02,  -1.89348738e-01,  -1.52778780e-02,
         -4.83470061e+03,  -2.24876825e-01,  -3.51404147e-02,
         -3.61380264e-02,  -1.44153854e-02,  -6.93153222e-02,
         -5.57776720e+03,   1.27661001e-01,   2.12529224e+04,
         -6.60236371e-02,  -7.43667177e+03,   2.57535318e+04,
          9.26983841e-01,   2.41042768e-02,  -4.98843836e+03,
         -7.80721318e-02,  -6.03409372e-02,  -4.91128984e+03,
        

In [124]:
def fit_model(x, ws, ids):
    y_pred = np.zeros(x.shape[0])
    #x=x[:,1:]
    jets_ind = jets_indices(x)
    x = standardize(x)
    
    for index, degree, w in zip(jets_ind, [2, 4, 5], ws):
        x_tr = x[index[0]][:,index[1]]

        x_poly = build_poly(x_tr, degree)
        x_poly = np.hstack((np.ones((x_poly.shape[0], 1)), x_poly))
        jet_pred = predict_labels(w, x_poly)
        print(jet_pred.shape)
        y_pred[index[0]] = jet_pred
        
    create_csv_submission(ids, y_pred, "submission_2fold1000.csv")

In [125]:
fit_model(test_data, predicted_w, ids_test)

(227458,)
(175338,)
(165442,)


In [126]:
import pickle

In [127]:
with open("best_weight.pickle", 'wb') as file:
    pickle.dump(predicted_w, file)